In [1]:
print('Starting generic python imports')
from pathlib import Path
import numpy as np
import os
from time import time, sleep
import sys
from munch import Munch
import random

print('Doing FKMC imports')
from FKMC.montecarlo import *

## overall input parameters

#put the repeat axis at the top so that I can watch the data come in in realtime!
repeats = 1
Js = np.linspace(0, 10.0, 30)
Ts = np.linspace(0.1, 5.0, 30)
U = 1
alpha = 1.25
Ns = [4,8,16,32,64,128]

MCMC_params = Munch(
        state = None, #this gets filled in later because it depends on N
        N_steps = int(1e5),
        N_burn_in = int(1e4), 
        thin = 100,
        logger = Eigenspectrum_IPR_all(bins = 1000, limit = 10),
        proposal = p_multi_site_uniform_reflect,
        accept_function = perturbation_accept,
        warnings = True,
    )


total_jobs = repeats * len(Js) * len(Ts)

#give information to the dispatch script
batch_params = Munch(total_jobs = total_jobs, Js = Js, Ts = Ts, Ns = Ns, repeats = repeats, structure_names = ('repeats','Js','Ts'))

print('Getting environment variables')
slurm_job_id = int(os.getenv('SLURM_ARRAY_JOB_ID', -1))
slurm_task_id = int(os.getenv('SLURM_ARRAY_TASK_ID', random.random()*total_jobs))
max_slurm_task_id = int(os.getenv('SLURM_ARRAY_TASK_MAX', total_jobs - 1))

##sleep if necessary
debug = (slurm_job_id == -1)
sleeptime = np.random.random() * (30)
print(f'Waiting for {sleeptime:.0f} seconds to randomise the finish time')
if not debug: sleep(sleeptime)

##using the value of slurm_task_id, decide what parameters this job will have
repeat_i, j = divmod(slurm_task_id, len(Js) * len(Ts))
J_i, T_i = divmod(j, len(Ts))

J = Js[J_i]
T = Ts[T_i]

logs = np.empty(shape = len(Ns), dtype = object)

if debug:
    print(f'''
{total_jobs}, {slurm_task_id}, {repeat_i}, {J_i}, {T_i}
    ''')
    
    MCMC_params.N_steps = 10
    MCMC_params.N_burn_in = 0
    MCMC_params.thin = 1

parameters = Munch(N_sites = 'variable', t = 1, alpha = alpha, mu = 0, beta = 1/T, J = J, U = U, normalise = True)
print(f'''
params = {parameters},
mc_params = {MCMC_params}
''')
    
for i, N in enumerate(Ns):
    print(f'starting N = {N}')
    parameters.N_sites = N
    
    t0 = time()
    logs[i] = FK_mcmc(**MCMC_params, parameters = parameters)
    logs[i].time = time() - t0
    


filepath = Path(f'./{slurm_task_id}.npz') if not debug else Path(f'./test_save.npz')

print(f'Saving in {filepath}')
np.savez_compressed(filepath, 
    **batch_params, parameters = parameters, MCMC_params = MCMC_params, logs = logs, allow_pickle = True,
    desc = ''
    )
print('done')

Starting generic python imports
Doing FKMC imports
Getting environment variables
Waiting for 6 seconds to randomise the finish time

900, 563, 0, 18, 23
    

params = Munch({'N_sites': 'variable', 't': 1, 'alpha': 1.25, 'mu': 0, 'beta': 0.2508650519031142, 'J': 6.206896551724139, 'U': 1, 'normalise': True}),
mc_params = Munch({'state': None, 'N_steps': 10, 'N_burn_in': 0, 'thin': 1, 'logger': <FKMC.montecarlo.Eigenspectrum_IPR_all object at 0x7f76d51c8828>, 'proposal': <function p_multi_site_uniform_reflect at 0x7f76bcb55268>, 'accept_function': <function perturbation_accept at 0x7f76bcb55598>, 'warnings': True})

starting N = 4
0% through after 0.00m
10% through after 0.00m
20% through after 0.00m
30% through after 0.00m
40% through after 0.00m
50% through after 0.00m
60% through after 0.00m
70% through after 0.00m
80% through after 0.00m
90% through after 0.00m
Warning, p_acc = 0.55, Munch({'N_sites': 4, 't': 1, 'alpha': 1.25, 'mu': 0, 'beta': 0.2508650519031142, 'J': 6.206896551724

In [ ]:
5